In [7]:
import json
import os
import pandas as pd
import unicodedata
import translitcodec
import codecs

In [2]:
MATCH_DATA_FILE = "match_data.csv"
SUMMARY_FILE = "summary.json"

input_dir = "../data/raw/"
all_dirs = os.listdir(input_dir)

csv_file = os.path.join(input_dir, all_dirs[0], MATCH_DATA_FILE)
json_file = os.path.join(input_dir, all_dirs[0], SUMMARY_FILE)
df = pd.read_csv(csv_file, index_col=0)
df

,player_1,player_2,player_3,player_4,player_5,player_6,player_7,player_8,player_9,player_10,...,goalkeeper_saves,fouls,yellow_cards,total_passes,completed_passes,tackles,attacks,dangerous_attacks,crosses_completed,goals
poland,bednarek j.,bochniewicz p.,frankowski p.,kiwior j.,lewandowski r.,piotrowski j.,slisz b.,swiderski k.,szczesny w.,szymanski d.,...,2.0,12.0,1.0,553.0,458.0,20.0,111.0,54.0,6.0,1
czech republic,brabec j.,chytil m.,coufal v.,doudera d.,holes t.,kuchta j.,provod l.,sadilek m.,soucek t.,stanek j.,...,4.0,10.0,1.0,361.0,265.0,12.0,103.0,44.0,4.0,1


In [3]:
with open(json_file, "r") as f:
    summary = json.load(f)
summary

{'teams': ['poland', 'czech republic'],
 'datetime': '2023-11-17T21:45:00',
 'poland': {'goals': [[38, 'piotrowski j.', None]],
  'substitutions': [[46, 'buksa a.', 'swiderski k.'],
   [58, 'peda p.', 'bochniewicz p.'],
   [73, 'grosicki k.', 'szymanski d.'],
   [85, 'szymanski s.', 'peda p.']],
  'yellow_cards': [[36, 'bochniewicz p.']],
  'red_cards': []},
 'czech republic': {'goals': [[49, 'soucek t.', None]],
  'substitutions': [[46, 'cvancara t.', 'chytil m.'],
   [46, 'hlozek a.', 'kuchta j.'],
   [75, 'lingr o.', 'provod l.'],
   [91, 'kral a.', 'sadilek m.']],
  'yellow_cards': [[63, 'brabec j.']],
  'red_cards': []}}

In [4]:
df.iloc[1, :]

player_1              brabec j.
player_2              chytil m.
player_3              coufal v.
player_4             doudera d.
player_5               holes t.
player_6              kuchta j.
player_7              provod l.
player_8             sadilek m.
player_9              soucek t.
player_10             stanek j.
player_11               zima d.
expected_goals             1.32
ball_possession            43.0
goal_attempts              15.0
shots_on_goal               3.0
shots_off_goal              9.0
blocked_shots               3.0
free_kicks                 16.0
corner_kicks                3.0
offsides                    0.0
goalkeeper_saves            4.0
fouls                      10.0
yellow_cards                1.0
total_passes              361.0
completed_passes          265.0
tackles                    12.0
attacks                   103.0
dangerous_attacks          44.0
crosses_completed           4.0
goals                         1
Name: czech republic, dtype: object

In [16]:
def _parse_player(line: str) -> str:
    player = line.split(',')[1].strip().lower()
    player_base = codecs.encode(player, 'translit/one').encode('ascii', 'replace').decode()
    if len(player_base) != len(player):
        raise AssertionError(f"len({player_base!r}) != len({player})")

    player_name = player_base.split()
    # swap name and surename, shorten name to first letter
    return f"{player_name[1]} {player_name[0][0]}."


def parse_squad(f) -> list[str]:
    for _ in range(SKIP_FIRST):
        _ = next(f)
    return [_parse_player(line) for line in f if not line.isspace()]        

In [29]:
SKIP_FIRST = 5
SKIP_LAST = 1
squad_dir = "../data/squads/"
first_team = df.index[0].replace(' ', '-')
second_team = df.index[1].replace(' ', '-')
with open(os.path.join(squad_dir, first_team) + ".txt", "r") as f:
    first_squad = parse_squad(f)
with open(os.path.join(squad_dir, second_team) + ".txt", "r") as f:
    second_squad = parse_squad(f)

print('-' * 79)
print(f"|{first_team.upper():^38}|{second_team.upper():^38}|")
print('-' * 79)
for p1, p2 in zip(first_squad, second_squad):
    print(f"|{p1:^38}|{p2:^38}|")
print('-' * 79)

-------------------------------------------------------------------------------
|                POLAND                |            CZECH-REPUBLIC            |
-------------------------------------------------------------------------------
|             szczesny w.              |              stanek j.               |
|             skorupski l.             |               kovar m.               |
|               bulka m.               |               jaros v.               |
|              salamon b.              |               zima d.                |
|            dawidowicz p.             |               holes t.               |
|            walukiewicz s.            |              hranac r.               |
|             bednarek j.              |              coufal v.               |
|              kiwior j.               |               vitik m.               |
|              puchacz t.              |              doudera d.              |
|            bereszynski b.            |

In [35]:
for _, team in df.iterrows():
    for player in team[[f"player_{i}" for i in range(1, 12)]]:
        if player not in first_squad and player not in second_squad:
            print(f"Player {player!r} not found")

Player 'bochniewicz p.' not found
Player 'brabec j.' not found


### Create feature set

In [ ]:
df_data = pd.DataFrame()